In [1]:
import torch
import torch_geometric

# 检查 PyTorch 版本和 CUDA 是否可用
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")


# 检查 PyG 版本
print(f"PyG version: {torch_geometric.__version__}")

# （可选）进一步检查稀疏操作库是否链接到正确的 CUDA 版本
try:
    import torch_scatter
    print("torch_scatter installed successfully.")
except ImportError:
    print("torch_scatter is not installed.")

try:
    import torch_sparse
    print("torch_sparse installed successfully.")
except ImportError:
    print("torch_sparse is not installed.")

PyTorch version: 2.8.0+cu128
CUDA available: True
CUDA version: 12.8
PyG version: 2.6.1
torch_scatter installed successfully.
torch_sparse installed successfully.


In [2]:
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.utils import dense_to_sparse

# 1. 原始数据 -> 节点特征矩阵 X
# 这是每个节点自身的属性。
# 形状: [num_nodes, num_node_features] -> [4, 4]
x = torch.tensor([
    [1, 0, 1, 0],
    [1, 1, 0, 1],
    [1, 1, 0, 0],
    [0, 1, 1, 1]
], dtype=torch.float)

# 2. 概念格导出的邻接矩阵 A+
# 这描述了节点之间的连接关系和强度。
# 形状: [num_nodes, num_nodes] -> [4, 4]
A_plus = torch.tensor([
    [6, 6, 5, 8],
    [6, 10, 8, 10],
    [5, 8, 6, 6],
    [8, 10, 6, 10]
], dtype=torch.float)

# 3. 将稠密邻接矩阵 A+ 转换为 PyG 的稀疏格式
# dense_to_sparse 会返回 edge_index 和 edge_attr (边权重)
# edge_index: 记录了边的起点和终点
# edge_attr: 记录了每条边的权重，值来自 A+
edge_index, edge_attr = dense_to_sparse(A_plus)

# 4. 创建 PyTorch Geometric 的 Data 对象
# 这是输入给GNN模型的标准数据结构，封装了图的所有信息。
graph_data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

# 打印看一下创建好的图数据对象
print("--- PyG Data Object ---")
print(graph_data)
print("\n节点特征矩阵 x (shape {}):".format(graph_data.x.shape))
print(graph_data.x)
print("\n边索引 edge_index (shape {}):".format(graph_data.edge_index.shape))
print(graph_data.edge_index)
print("\n边权重 edge_attr (shape {}):".format(graph_data.edge_attr.shape))
print(graph_data.edge_attr)


# 5. (演示) 定义一个简单的GNN模型并输入数据
class SimpleGNN(torch.nn.Module):
    def __init__(self, num_features, hidden_channels, num_classes):
        super().__init__()
        # GCNConv 支持接收 edge_weight 参数
        self.conv1 = GCNConv(num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, num_classes)

    def forward(self, x, edge_index, edge_weight):
        # 第一层卷积
        x = self.conv1(x, edge_index, edge_weight)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        # 第二层卷积
        x = self.conv2(x, edge_index, edge_weight)
        return x

# 实例化模型 (假设最终要分为3类)
model = SimpleGNN(num_features=4, hidden_channels=16, num_classes=3)
print("\n--- GNN Model ---")
print(model)

# 将打包好的 graph_data 输入模型
# 注意，我们需要从 data 对象中解包各个属性来传入 forward 函数
output = model(graph_data.x, graph_data.edge_index, graph_data.edge_attr)

print("\n--- GNN Output ---")
print("模型输出的节点表示 (shape {}):".format(output.shape))
print(output)

--- PyG Data Object ---
Data(x=[4, 4], edge_index=[2, 16], edge_attr=[16])

节点特征矩阵 x (shape torch.Size([4, 4])):
tensor([[1., 0., 1., 0.],
        [1., 1., 0., 1.],
        [1., 1., 0., 0.],
        [0., 1., 1., 1.]])

边索引 edge_index (shape torch.Size([2, 16])):
tensor([[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3],
        [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3]])

边权重 edge_attr (shape torch.Size([16])):
tensor([ 6.,  6.,  5.,  8.,  6., 10.,  8., 10.,  5.,  8.,  6.,  6.,  8., 10.,
         6., 10.])

--- GNN Model ---
SimpleGNN(
  (conv1): GCNConv(4, 16)
  (conv2): GCNConv(16, 3)
)

--- GNN Output ---
模型输出的节点表示 (shape torch.Size([4, 3])):
tensor([[-0.6080,  0.9541, -0.0057],
        [-0.6341,  1.1244,  0.0627],
        [-0.5077,  1.0186,  0.0631],
        [-0.7182,  1.1266,  0.0242]], grad_fn=<AddBackward0>)
